# MLP as per Wang, Yan and Oates

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline

See MIT License in https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 Online.


In [ ]:
#!/usr/bin/env python3

import os
from pathlib import Path
import time
from datetime import datetime
from dateutil.tz import gettz
import itertools

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
from sklearn.model_selection import KFold, RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, classification_report

np.random.seed(813306)

# User inputs

In [ ]:
flist = ['private_correct_plus'] #, ['private_dog0_correct_plus'] # List of dataset directory names. WormsTwoClass Lightning2 Earthquakes GunPoint 
batch_size = -1 # Set to -1 to use Wang et al settings 
nb_epochs = 5000 # Wang et al. setting is 5000 but min loss is usually found at <1500
k = 3 # For k-fold cross validation. If k=1, the original test-train split is used.
m = 4 # Number of repetitions of k-fold cross validation (if k>1).
early_stopping = False 
tensorboard = True # Set to True to write logs for use by TensorBoard
k_fold_seed = 87

# Output directories
logs_dir = '../logs'
tensorboard_dir = '../logs/tensorboard'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now(gettz("Europe/London")))
logs_dir = logs_dir +'/' + timestamp
tensorboard_dir = tensorboard_dir +'/' + timestamp

# Input directory
if 'private' in flist[0]:
    fdir = '../data/private_data/private_events_dev2' 
else:
    fdir = '../data' 

# Tools

In [ ]:
def plot_confusion_matrix(cm, title='Normalised confusion matrix', save=False):
    ''' Plot the normalised confusion matrix
    Parameters
    cm : array - normalised confusion matrix
    Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.
    'Confusion Matrix' https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
    '''
    classes = ['Positive', 'Negative']
    cmap=plt.cm.Blues
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.clim(0, 1)
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.tight_layout()
    if save:
        plt.savefig('cm_mlpwang.png', bbox_inches='tight')
        
        
def plot_roc(y_true, y_probs, save=False): 
    ''' Plot ROC and return AUC
    Parameters
    y_true : vector of true class labels.
    y_probs : array of predicted probabilities, one column for each class.
    Returns
    auc : float
    '''
    fpr, tpr, thresholds = roc_curve(y_true, y_probs[:,1])
    auc = roc_auc_score(y_true, y_probs[:,1])
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic curve')
    plt.legend(loc="lower right")
    plt.show()
    if save:
        plt.savefig('roc_mlwang.png', bbox_inches='tight')
    return auc


def readucr(filename):
    ''' Load a dataset from a file in UCR format
    space delimited, class labels in the first column.
    Returns
    X : DNN input data
    Y : class labels
    '''
    data = np.loadtxt(Path(filename))
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
   

## MLP

In [ ]:
def build_mlp(input_shape, nb_classes):
    num = 500
    x = Input(shape=(input_shape))
    y = Dropout(0.1,name='WDrop010')(x)
    y = Dense(num, activation='relu', name='WDense010')(y)
    y = Dropout(0.2,name='WDrop020')(y)
    y = Dense(num, activation='relu', name='WDense020')(y)
    y = Dropout(0.2,name='WDrop021')(y)
    y = Dense(num, activation='relu', name='WDense021')(y)
    y = Dropout(0.3,name='WDrop031')(y)
    out = Dense(nb_classes, activation='softmax', name='WDense080')(y)
    return x, out 

# Train model

In [ ]:
def train_model(fname, x_train, y_train, x_test, y_test, label="0"):
    print('Running dataset', fname)
    if batch_size == -1:
        batch = int(min(x_train.shape[0]/10, 16)) # Wang et al. setting.
    else:
        batch=batch_size 
        
    nb_classes = len(np.unique(y_test))
     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
     
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std) 
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    x, y = build_mlp(x_train.shape[1:], nb_classes)
    model = Model(x, y)
    #print(model.summary())
    Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    
    optimizer = keras.optimizers.Adadelta(rho=0.95, epsilon=1e-8) # paper specifies rho and epsilon, although GitHub code does not. 
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
                      patience=200, min_lr=0.1)
    callbacks = [reduce_lr]
    if tensorboard:
        tb_dir = tensorboard_dir+'/'+fname+'_'+label
        Path(tb_dir).mkdir(parents=True, exist_ok=True) 
        print('Tensorboard logs in', tb_dir)
        callbacks.append(keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0))
  
    start = time.time()
    print(x_test[:10])
    print(Y_test[:10])
    hist = model.fit(x_train, Y_train, batch_size=batch, epochs=nb_epochs,
             verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
    end = time.time()
    log = pd.DataFrame(hist.history) 
    
    # Print results
    duration_seconds = round(end-start)
    duration_minutes = str(round((end-start)/60))
    print('Training complete on', fname, 'Duration:', duration_seconds, 'secs; about', duration_minutes, 'minutes.')
    
    # Print and save results. Print the testing results which has the lowest training loss.
    print('Selected the test result with the lowest training loss. Loss and validation accuracy are -')
    idx = log['loss'].idxmin()
    loss = log.loc[idx]['loss']
    val_acc = log.loc[idx]['val_acc']
    epoch = idx + 1
    print(loss, val_acc, 'at index', str(idx), ' (epoch ', str(epoch), ')')   
    summary = '|' + label + '  |'+str(loss)+'  |'+str(val_acc)+' |'+str(epoch)+' |'+ duration_minutes + 'mins  |'
    summary_csv = label+','+str(loss)+','+str(val_acc)+','+str(epoch)+','+ duration_minutes 
    
    # Save summary file and log file.
    print('Tensorboard logs in', tb_dir)
    with open(logs_dir+'/'+fname+'/mlpwang_summary.csv', 'a+') as f:
        f.write(summary_csv)
        f.write('\n')
        print('Added summary row to ', logs_dir+'/'+fname+'/mlpwang_summary.csv')  
    print('Saving logs to',logs_dir+'/'+fname+'/history_'+label+'.csv')
    log.to_csv(logs_dir+'/'+fname+'/history_'+label+'.csv')
    
    return summary, model

# Train DNN

In [ ]:
results = []
for each in flist:
    fname = each
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    # k-fold cross validation setup
    if k > 1:
        x_all = np.concatenate((x_train, x_test), axis=0)
        y_all = np.concatenate((y_train, y_test), axis=0)
        kfold = RepeatedStratifiedKFold(n_splits=k, n_repeats=m, random_state=k_fold_seed)
        count = 0
        for train, test in kfold.split(x_all, y_all):
            x_train, y_train, x_test, y_test = x_all[train], y_all[train], x_all[test], y_all[test]
            summary, model = train_model(fname, x_train, y_train, x_test, y_test, str(count))
            results.append(summary)
            count = count + 1
    else:
        summary, model = train_model(fname, x_train, y_train, x_test, y_test)
        results.append(summary)
        
print('DONE')
print(fname, timestamp)
print('train:test', y_train.shape[0], y_test.shape[0])
for each in results:
    print(each)

In [ ]:
# Print when done
print('Done at:' , '{:%Y-%m-%dT%H:%M}'.format(datetime.now()))

# Metrics

In [ ]:
# Use trained model (after all epochs) to make predictions
do_print = True
x_input = x_test
y_input = y_test
y_input = y_input - y_input.min()
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_input = (x_input - x_train_mean)/(x_train_std)
nb_classes = len(np.unique(y_input))
y_input = (y_input - y_input.min())/(y_input.max()-y_input.min())*(nb_classes-1)
# Class balance
n0 = (y_input == 0).sum()
n1 = (y_input == 1).sum()
# Calculate model prediction
y_probs = model.predict_on_batch(x_input)
y_class = y_probs.argmax(axis=1)
cm = confusion_matrix(y_input, y_probs.argmax(axis=1), labels=[1,0])
acc_calc = (cm[0][0]+cm[1][1])/(cm.sum())
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
if do_print:
    print('Predicted class probabilities:\n', y_probs[:5,:])
    print('Pred', y_class[:20])
    print('True', y_input[:20].astype(int))
    print(cm)
    print('Calculated accuracy:',acc_calc)
    print('Class balance in test set:', n0, 'to', n1, 'i.e.', n0/(n0+n1))
    print('Normalised confusion matrix:\n', cm_norm)
title = 'Normalised confusion matrix'
plot_confusion_matrix(cm_norm, title=title, save=True)

# ROC and AUC
auc = plot_roc(y_input, y_probs)
print('AUC:', auc)

report = classification_report(y_input, y_class)
print('\n', report)
print('\nmicro av - averaging the total true positives, false negatives and false positives')
print('macro av - averaging the unweighted mean per label')
print('weighted av - averaging the support-weighted mean per label')

# Quantiles

In [ ]:
file =  logs_dir+'/'+fname+'/mlpwang_summary.csv'
data = pd.read_csv(file, header=None, names=['run','loss','val_acc','epoch','time'])
accuracy = data['val_acc']
print(file)
print('Accuracy mean, sample std dev and 95% confidence level is', accuracy.mean(), accuracy.std(), accuracy.std()*2.262)
print('95% quantile interval is', accuracy.quantile(0.0025), 'to', accuracy.quantile(0.975))
data.boxplot(column=['val_acc'], whis=[2.5,97.5])

## Make predictions on other datasets

In [ ]:
other = fname+'_END_TEST' #_dog_incorrect' # 'private_dog0_correct_plus_END_TEST'
datadir = fdir+'/'+fname
print('Testing on:', datadir+'/'+other+'.txt')
print('Testing model that has validation accuracy', accuracy[9])

x_other, y_other = readucr(datadir+'/'+other+'.txt')

x_input = x_other
y_input = y_other
y_input = y_input - y_input.min()
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_input = (x_input - x_train_mean)/(x_train_std)
nb_classes = len(np.unique(y_input))
y_input = (y_input - y_input.min())/(y_input.max()-y_input.min())*(nb_classes-1)
# Calculate model prediction
y_probs = model.predict_on_batch(x_input)
y_class = y_probs.argmax(axis=1)
cm = confusion_matrix(y_input, y_probs.argmax(axis=1), labels=[1,0])
acc_calc = (cm[0][0]+cm[1][1])/(cm.sum())
print(cm)
print('accuracy', acc_calc)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
title = 'Normalised confusion matrix on end test dataset'
plot_confusion_matrix(cm_norm, title=title, save=True)